In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import numpy as np

class Block(nn.Module):
    '''expand + depthwise + pointwise + squeeze-excitation'''

    def __init__(self, in_planes, out_planes, expansion, stride):
        super(Block, self).__init__()
        self.stride = stride

        planes = expansion * in_planes
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, groups=planes, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(
            planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(out_planes)

        self.shortcut = nn.Sequential()
        if stride == 1 and in_planes != out_planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, out_planes, kernel_size=1,
                          stride=1, padding=0, bias=False),
                nn.BatchNorm2d(out_planes),
            )

        # SE layers
        self.fc1 = nn.Conv2d(out_planes, out_planes//16, kernel_size=1)
        self.fc2 = nn.Conv2d(out_planes//16, out_planes, kernel_size=1)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        shortcut = self.shortcut(x) if self.stride == 1 else out
        # Squeeze-Excitation
        w = F.avg_pool2d(out, out.size(2))
        w = F.relu(self.fc1(w))
        w = self.fc2(w).sigmoid()
        out = out * w + shortcut
        return out


class EfficientNet(nn.Module):
    def __init__(self, cfg, num_classes=10):
        super(EfficientNet, self).__init__()
        self.cfg = cfg
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_planes=32)
        self.linear = nn.Linear(cfg[-1][1], num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for expansion, out_planes, num_blocks, stride in self.cfg:
            strides = [stride] + [1]*(num_blocks-1)
            for stride in strides:
                layers.append(Block(in_planes, out_planes, expansion, stride))
                in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def EfficientNetB0():
    # (expansion, out_planes, num_blocks, stride)
    cfg = [(1,  16, 1, 2),
           (6,  24, 2, 1),
           (6,  40, 2, 2),
           (6,  80, 3, 2),
           (6, 112, 3, 1),
           (6, 192, 4, 2),
           (6, 320, 1, 2)]
    return EfficientNet(cfg)


def test():
    net = EfficientNetB0()
    x = torch.randn(2, 3, 32, 32)
    y = net(x)
    print(y.shape)

In [0]:
def utility(corruption_per, nos_epoch):

  lr = 0.01

  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  best_acc = 0  # best test accuracy
  start_epoch = 0  # start from epoch 0 or last checkpoint epoch

  # Data
  print('==> Preparing data..')
  transform_train = transforms.Compose([
      transforms.RandomCrop(32, padding=4),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])

  transform_test = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])

  trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
  testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)


  # *******************************************************************************
  if corruption_per != 0:
    np.random.seed(1234)
    mask = np.random.uniform(0,1,50000) < corruption_per

    a = np.array(trainset.targets)
    print("true",a[mask])
    print(np.sum(mask))
    a[mask] = np.random.randint(0,10,sum(mask))
    print("randomized",a[mask])
    trainset.targets = list(a)
  #**********************************************************************************

  testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=False, num_workers=2)

  classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
  # Model
  print('==> Building model..')
  net = EfficientNetB0()
  net = net.to(device)
  if device == 'cuda':
      net = torch.nn.DataParallel(net)
      cudnn.benchmark = True
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr, momentum=0.9, weight_decay=5e-4)
  
  def accuracy( a, b):
    length = a.shape
    correct = a==b
    return sum(correct)/length
  
  def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print("train accuracy ", correct/total ,train_loss/391)
    
  def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    print("test accuracy ", correct/total , test_loss/100)
    
  def print_train():
    net.eval()
    train_loss = 0
    correct = 0
    total = 0
    true = []
    pred=[]
    out =[]
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(trainloader):
          inputs, targets = inputs.to(device), targets.to(device)
          true.append(targets.cpu().numpy())

          outputs = net(inputs)
          out.append(outputs.cpu())
          loss = criterion(outputs, targets)


          train_loss += loss.item()
          _, predicted = outputs.max(1)
          pred.append(predicted.cpu().numpy())
          total += targets.size(0)
          correct += predicted.eq(targets).sum().item()

        print("train accuracy ", correct/total ,train_loss/391)

    true_targets = np.concatenate(true,axis=0)
    predicted_targets = np.concatenate(pred,axis =0)

    print("---"*20)
    print("Train accuracy on corrupt data",accuracy(true_targets[mask], predicted_targets[mask]))
    print("Train accuracy on un-corrupt data",accuracy(true_targets[~mask], predicted_targets[~mask]))
    print("Train accuracy on full data", accuracy(true_targets, predicted_targets))

    # print( sum (predicted_targets == np.argmax(out, axis =1)))

    l= np.where(mask ==True)
    p = np.where(mask == False)

    out = torch.cat(out, dim =0)

    print("Train cross entropy loss on corrupt data", criterion(out[l], torch.Tensor(true_targets[l]).type(torch.LongTensor)))
    print("Train cross entropy loss on un-corrupt data",criterion(out[p], torch.Tensor(true_targets[p]).type(torch.LongTensor)))
    print("Train cross entropy loss on full data",criterion(out, torch.Tensor(true_targets).type(torch.LongTensor)))
    print("---"*20)
    
  def print_test( corrupt_per): 
    testset_len = len(testset.targets)
    np.random.seed(1234)
    mask1 = np.random.uniform(0,1,testset_len) < corrupt_per

    a = np.array(testset.targets)
    print("true",a[mask1])
    print(np.sum(mask1))
    a[mask1] = np.random.randint(0,10,sum(mask1))
    print("randomized",a[mask1])
    testset.targets = list(a)

    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    true = []
    pred=[]
    out =[]
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
          inputs, targets = inputs.to(device), targets.to(device)
          true.append(targets.cpu().numpy())

          outputs = net(inputs)
          out.append(outputs.cpu())
          loss = criterion(outputs, targets)


          test_loss += loss.item()
          _, predicted = outputs.max(1)
          pred.append(predicted.cpu().numpy())
          total += targets.size(0)
          correct += predicted.eq(targets).sum().item()

        print("test accuracy ", correct/total ,test_loss/100)

    true_targets = np.concatenate(true,axis=0)
    predicted_targets = np.concatenate(pred,axis =0)
    print("---"*20)
    print("Test accuracy on corrupt data",accuracy(true_targets[mask1], predicted_targets[mask1]))
    print("Test accuracy on un-corrupt data",accuracy(true_targets[~mask1], predicted_targets[~mask1]))
    print("Test accuracy on full data", accuracy(true_targets, predicted_targets))

    l = np.where(mask1 ==True)
    p  = np.where(mask1 == False)
    out = torch.cat(out, dim =0)

    print("Test cross entropy loss on corrupt data", criterion(out[l], torch.Tensor(true_targets[l]).type(torch.LongTensor)))
    print("Test cross entropy loss on un-corrupt data",criterion(out[p], torch.Tensor(true_targets[p]).type(torch.LongTensor)))
    print("Test cross entropy loss on full data",criterion(out, torch.Tensor(true_targets).type(torch.LongTensor)))
    print("---"*20)

    
  for epoch in range(start_epoch, start_epoch + nos_epoch):
    train(epoch)
    test(epoch)
    
  torch.save(net.state_dict(),"model_"+str(corruption_per)+"_"+str(nos_epoch))
  if corruption_per != 0:
    print_train()
    print_test(corruption_per)

In [0]:
# def main_function(corrupt_per, nos_epoch):

#   utility(corrupt_per)
  
  

In [0]:
# def zero_corrupt(nos_epoch):
#   utility(0)
#   for epoch in range(start_epoch, start_epoch + nos_epoch):
#     train(epoch)
#     test(epoch)
  
#   torch.save(net.state_dict(),"model_0_"+str(nos_epoch))

In [0]:
utility(0 , 200)


Epoch: 0



170500096it [00:29, 16517540.39it/s]                               

train accuracy  0.39066 1.6516172026124452
test accuracy  0.4808 1.4229543280601502

Epoch: 1
train accuracy  0.51132 1.3409577562376056
test accuracy  0.5782 1.1665966618061065

Epoch: 2
train accuracy  0.57696 1.1718556249842924
test accuracy  0.6312 1.03860844373703

Epoch: 3
train accuracy  0.6238 1.0455082261653812
test accuracy  0.6669 0.9347455018758773

Epoch: 4
train accuracy  0.66328 0.9456100588869256
test accuracy  0.6942 0.8558773124217987

Epoch: 5
train accuracy  0.6885 0.8737364446415621
test accuracy  0.6991 0.8642270648479462

Epoch: 6
train accuracy  0.70874 0.818369796971226
test accuracy  0.7384 0.7581718671321869

Epoch: 7
train accuracy  0.73028 0.7633983325165556
test accuracy  0.747 0.7280357950925826

Epoch: 8
train accuracy  0.74368 0.7204524675750976
test accuracy  0.7528 0.6958274725079536

Epoch: 9
train accuracy  0.75932 0.6797146039545688
test accuracy  0.7706 0.6540326023101807

Epoch: 10
train accuracy  0.77092 0.6502089864000336
test accuracy  0.7807 

In [0]:
utility(0.1, 200)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
true [3 6 5 ... 6 7 7]
5054
randomized [9 1 5 ... 0 0 4]
==> Building model..

Epoch: 0
train accuracy  0.34106 1.8549894521303494
test accuracy  0.4824 1.4168539118766785

Epoch: 1
train accuracy  0.45138 1.6066841987697669
test accuracy  0.5595 1.2296291041374205

Epoch: 2
train accuracy  0.50378 1.4876861932027676
test accuracy  0.604 1.1349084603786468

Epoch: 3
train accuracy  0.54304 1.39981737313673
test accuracy  0.625 1.0745630580186845

Epoch: 4
train accuracy  0.5709 1.3323898659947584
test accuracy  0.6535 0.9968793135881424

Epoch: 5
train accuracy  0.59318 1.2784944707170471
test accuracy  0.6811 0.928900750875473

Epoch: 6
train accuracy  0.61268 1.2311373915513764
test accuracy  0.6909 0.9064959824085236

Epoch: 7
train accuracy  0.62954 1.1878009822667408
test accuracy  0.7145 0.8411756789684296

Epoch: 8
train accuracy  0.64222 1.1578997234859125
test accuracy  0.732 0.803

In [0]:
utility(0.2, 220)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
true [6 3 6 ... 8 7 7]
10054
randomized [9 1 5 ... 3 2 6]
==> Building model..

Epoch: 0
train accuracy  0.29484 1.9996504664725965
test accuracy  0.4457 1.529771136045456

Epoch: 1
train accuracy  0.38248 1.8188931411489502
test accuracy  0.5108 1.4188989698886871

Epoch: 2
train accuracy  0.42484 1.7355078821596892
test accuracy  0.55 1.314424422979355

Epoch: 3
train accuracy  0.45692 1.6702284166575088
test accuracy  0.5815 1.2280953401327133

Epoch: 4
train accuracy  0.48054 1.6144267494416298
test accuracy  0.611 1.1627164769172669

Epoch: 5
train accuracy  0.50796 1.5634738164179771
test accuracy  0.6393 1.081918345093727

Epoch: 6
train accuracy  0.52662 1.518859106866295
test accuracy  0.6647 1.035222037434578

Epoch: 7
train accuracy  0.54466 1.4816901793565287
test accuracy  0.6861 0.9782383990287781

Epoch: 8
train accuracy  0.5606 1.4459634839421343
test accuracy  0.6982 0.9490

In [3]:
utility(0.3, 240)

0it [00:00, ?it/s]

==> Preparing data..


 99%|█████████▉| 169484288/170498071 [00:13<00:00, 14236256.41it/s]

Files already downloaded and verified
true [6 1 2 ... 1 0 6]
15024
randomized [9 1 5 ... 3 7 8]
==> Building model..

Epoch: 0


170500096it [00:29, 14236256.41it/s]                               

train accuracy  0.25948 2.1007714548989025
test accuracy  0.4136 1.644756418466568

Epoch: 1
train accuracy  0.34086 1.9495724970117554
test accuracy  0.5017 1.456616678237915

Epoch: 2
train accuracy  0.3765 1.8808049076353497
test accuracy  0.5542 1.3565058660507203

Epoch: 3
train accuracy  0.40732 1.8246450268703958
test accuracy  0.583 1.2792891764640808

Epoch: 4
train accuracy  0.43014 1.783652424812317
test accuracy  0.6093 1.2425182747840882

Epoch: 5
train accuracy  0.45034 1.7413991302480478
test accuracy  0.6173 1.2164178228378295

Epoch: 6
train accuracy  0.46398 1.7131102673537897
test accuracy  0.6472 1.1488155937194824

Epoch: 7
train accuracy  0.47724 1.6874286418070878
test accuracy  0.6321 1.1840512824058533

Epoch: 8
train accuracy  0.48962 1.6636129288417298
test accuracy  0.6718 1.0990068924427032

Epoch: 9
train accuracy  0.50096 1.6423385975610874
test accuracy  0.6781 1.0755514055490494

Epoch: 10
train accuracy  0.50906 1.6246763425105064
test accuracy  0.6981

In [0]:
utility(0.4, 260)

0it [00:00, ?it/s]

==> Preparing data..


100%|█████████▉| 170057728/170498071 [00:12<00:00, 15971270.25it/s]

Files already downloaded and verified
true [6 1 2 ... 1 0 6]
19976
randomized [9 1 5 ... 1 0 8]
==> Building model..

Epoch: 0


170500096it [00:30, 15971270.25it/s]                               

train accuracy  0.22032 2.1866669115203115
test accuracy  0.3814 1.7811587047576904

Epoch: 1
train accuracy  0.29086 2.0673729473977445
test accuracy  0.4535 1.6426325130462647

Epoch: 2
train accuracy  0.3191 2.0178972513169584
test accuracy  0.5013 1.5264356005191804

Epoch: 3
train accuracy  0.34172 1.985918422794098
test accuracy  0.5399 1.4507979083061218

Epoch: 4
train accuracy  0.36212 1.9505179269844308
test accuracy  0.5826 1.3795557987689973

Epoch: 5
train accuracy  0.37872 1.9185328559802317
test accuracy  0.5897 1.3469338476657868

Epoch: 6
train accuracy  0.39692 1.8928185515391551
test accuracy  0.6218 1.2806727492809296

Epoch: 7
train accuracy  0.409 1.8670090494863212
test accuracy  0.6282 1.2584942495822906

Epoch: 8
train accuracy  0.41878 1.8488994135576136
test accuracy  0.6647 1.1922413742542266

Epoch: 9
train accuracy  0.43018 1.8296352338303081
test accuracy  0.66 1.1997375619411468

Epoch: 10
train accuracy  0.43664 1.816453398036225
test accuracy  0.6724 1

In [4]:
utility(0.5, 280)

0it [00:00, ?it/s]

==> Preparing data..


170500096it [00:02, 69880849.24it/s]                               


Files already downloaded and verified
true [6 9 1 ... 0 6 9]
24980
randomized [9 1 5 ... 4 1 5]
==> Building model..

Epoch: 0
train accuracy  0.18728 2.2455544837600434
test accuracy  0.3836 1.8246359813213349

Epoch: 1
train accuracy  0.24606 2.161108540756928
test accuracy  0.4169 1.769227614402771

Epoch: 2
train accuracy  0.26808 2.1282263616161883
test accuracy  0.475 1.6585382580757142

Epoch: 3
train accuracy  0.28346 2.1018555121653524
test accuracy  0.5108 1.5814004909992219

Epoch: 4
train accuracy  0.30344 2.0769928480353195
test accuracy  0.531 1.5594814014434815

Epoch: 5
train accuracy  0.3104 2.059787242004024
test accuracy  0.5571 1.5145845746994018

Epoch: 6
train accuracy  0.32324 2.0445450029104872
test accuracy  0.571 1.4809049606323241

Epoch: 7
train accuracy  0.33336 2.02694986634852
test accuracy  0.5842 1.45162278175354

Epoch: 8
train accuracy  0.34042 2.012896850895699
test accuracy  0.6031 1.4239289617538453

Epoch: 9
train accuracy  0.3499 2.00168260771905